# Stabilizatión of the simulation

In [1]:
## -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

temp = 9.5
pressure = 0.1

#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_9.5_0.1_ramp.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 4.0, tauP = 3.5, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.log"
gsd_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
*

In [2]:
hoomd.run(3e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 2224565 / 2524000 | TPS 56.4663 | ETA 01:28:22
Time 00:00:20 | Step 2225109 / 2524000 | TPS 54.3823 | ETA 01:31:36
Time 00:00:30 | Step 2225761 / 2524000 | TPS 65.0778 | ETA 01:16:22
Time 00:00:40 | Step 2226372 / 2524000 | TPS 60.9719 | ETA 01:21:21


**ERROR**: nlist.tree(): Particle 263 is out of bounds (x: -273.839, y: -133.874, z: 159.699, fx: -3.4191, fy: -1.41597, fz:2.78557)


RuntimeError: Error updating neighborlist